GCP GET CLI Details

In [ ]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
from google.oauth2 import service_account
from googleapiclient import discovery

load_dotenv()
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")   # e.g. path/to/service-account.json
PROJECT  = os.getenv("GCP_PROJECT") 
print(f"Using project: {PROJECT}")
print(f"Using key: {KEY_PATH}")

Using project: my-gcp-project
Using key: /Users/alice/keys/my-gcp-key.json


In [ ]:
creds   = service_account.Credentials.from_service_account_file(KEY_PATH)
compute = discovery.build("compute", "v1", credentials=creds)

In [3]:
def collect_all_machine_types():
    rows = []
    request = compute.machineTypes().aggregatedList(project=PROJECT)
    while request:
        resp = request.execute()
        for zone_uri, zone_block in resp.get("items", {}).items():
            # zone_uri looks like "zones/us-central1-a"
            zone   = zone_uri.split("/")[-1]
            region = zone.rsplit("-", 1)[0]          # us-central1
            for m in zone_block.get("machineTypes", []):
                rows.append({
                    "Zone":            zone,
                    "Region":          region,
                    "MachineType":     m["name"],
                    "vCPUs":           m["guestCpus"],
                    "MemoryMiB":       m["memoryMb"],
                    "MemoryGiB":       round(m["memoryMb"]/1024, 2),
                    "Accelerators":    m.get("accelerators"),        # GPUs/FPGAs
                    "Deprecated":      m.get("deprecated"),          # if any
                    "SelfLink":        m.get("selfLink"),
                    "Description":     m.get("description"),
                })
        request = compute.machineTypes().aggregatedList_next(
            previous_request=request, previous_response=resp
        )
    return pd.DataFrame(rows)

In [ ]:
df = collect_all_machine_types()
print(df.columns.tolist())
print(df.head())